<a href="https://colab.research.google.com/github/akamezoya/RNN-Excercises/blob/main/Copy_of_RNN_HateSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network

*   Nama : Aji Purnomo
*   NPM : 197064416091
*   Tema : Hate speech monitorting with RNN
*   Deskripsi : Memonitoring hate coment menggunakan algoritma RNN
*   Tools : Google Colab (Karena laptop saya kentang)
*   Dataset Train : https://github.com/akamezoya/RNN-HateSpeech/tree/main 
*   Sumber Dataset Ori : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data

Note : Akun github dengan username Akamezoya merupakan akun github saya (Aji Purnomo) sendiri. Untuk memastikan bisa cek di profile github nya (Cek nama saya) atau kontak ke akamezoya@gmail.com ...



In [2]:
import pandas as pd
import itertools
import h5py
import pickle
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Model, Input, Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, SpatialDropout1D, Activation
from keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [3]:
in_folder='data/3-processed_data'
out_folder='models'

In [4]:
X_train=pd.read_csv('https://raw.githubusercontent.com/akamezoya/RNN-HateSpeech/main/X_train.csv')
Y_train=pd.read_csv('https://raw.githubusercontent.com/akamezoya/RNN-HateSpeech/main/Y_train.csv')
X_test=pd.read_csv('https://raw.githubusercontent.com/akamezoya/RNN-HateSpeech/main/X_test.csv')
Y_test=pd.read_csv('https://raw.githubusercontent.com/akamezoya/RNN-HateSpeech/main/Y_test.csv')
labels=list(Y_test.columns)

In [5]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,670,4510,12133,1115,87,331,52,2269,11510,51,6926,16,61,2747,149,8,2933,35,116,1235,15747,2859,5,46,60,243,1,370,32,1,39,29,144,74,3474,90,3075,4630,2284,985
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,53,2639,14,553,3803,74,4558,2699,22,95,39,807,2688,986,589,8493,183
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,74,134,15,250,3,73,315,79,51,10,14,625,9,2274,487,499,103,5,609,3,37,325,128,363,4,28,39,29,53,207,3,436,58,36,1,2402,94,1,733,468
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,267,37,70,2,2,42,539,3,17,6,6000,16,83,13,397,38,8,642,42,86,17,6,2023,385,6,3060,3243,77,79,425,11,1,499,639,3594,31,99,24,3557,4973
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1703,20,28,3595,55,1064,7,580,40,29,205,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159566,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,82,21,324,561,5857,1,1626,11,265,112,75,57,256,436,40,7,213,7,229,96,261,6,1909,618,9,1,1866,78,934,3,404,950,2197,26,1,136,950,778,2,2
159567,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,57,17,4655,4,208,2,2,10,9,6,3326,232,7,200,16,28,39,29,3485,3642,804,3475
159568,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,7356,5219,47,733,24,13,8217,3507,13781,4527
159569,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,12,568,50,12,25,209,7,63,200,16,1,255,3,19,1,114,396,137,90,10,8,152,35,12


In [6]:
# number of unique words we want to use (or: number of rows in incoming embedding vector)
max_features = 20000 

# max number of words in a comment to use (or: number of columns in incoming embedding vector)
max_len = 200 

# dimension of the embedding variable (or: number of rows in output of embedding vector)
embedding_dims = 128

In [7]:
# instantiate RNN model
rnn_model = Sequential()

# add embedding layer 
rnn_model.add(Embedding(input_dim=max_features, input_length=max_len,
                        output_dim=embedding_dims))

# set the dropout layer to drop out 50% of the nodes
rnn_model.add(SpatialDropout1D(0.5))

# add bidirectional layer and pass in an LSTM()
rnn_model.add(Bidirectional(LSTM(25, return_sequences=True)))

# add normalization layer
rnn_model.add(BatchNormalization())

# add pooling layer 
rnn_model.add(GlobalMaxPool1D())

# set the dropout layer to drop out 50% of the nodes
rnn_model.add(Dropout(0.5))

# add dense layer to produce an output dimension of 50 and using relu activation
rnn_model.add(Dense(50, activation='relu'))

# finally add a dense layer
rnn_model.add(Dense(7, activation='sigmoid'))

In [8]:
rnn_model.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.01),
                  metrics=['accuracy'])

rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          2560000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 128)         0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 200, 50)          30800     
 l)                                                              
                                                                 
 batch_normalization (BatchN  (None, 200, 50)          200       
 ormalization)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                         

In [9]:
rnn_hist = rnn_model.fit(X_train, Y_train, batch_size=128, 
                          epochs=5, validation_split=0.2)

Epoch 1/5
998/998 [==============================] - 402s 398ms/step - loss: 0.0822 - accuracy: 0.9452 - val_loss: 0.0609 - val_accuracy: 0.9578
Epoch 2/5
998/998 [==============================] - 396s 397ms/step - loss: 0.0622 - accuracy: 0.9567 - val_loss: 0.0588 - val_accuracy: 0.9587
Epoch 3/5
998/998 [==============================] - 399s 399ms/step - loss: 0.0591 - accuracy: 0.9589 - val_loss: 0.0593 - val_accuracy: 0.9588
Epoch 4/5
998/998 [==============================] - 395s 396ms/step - loss: 0.0580 - accuracy: 0.9599 - val_loss: 0.0639 - val_accuracy: 0.9589
Epoch 5/5
998/998 [==============================] - 402s 402ms/step - loss: 0.0573 - accuracy: 0.9605 - val_loss: 0.0603 - val_accuracy: 0.9587


In [10]:
rnn_test_loss, rnn_test_acc = rnn_model.evaluate(X_test, Y_test, batch_size=128)
print('Test Loss:    ', rnn_test_loss)
print('Test Accuracy:', rnn_test_acc)

500/500 [==============================] - 52s 104ms/step - loss: 0.0925 - accuracy: 0.9227
Test Loss:     0.09248067438602448
Test Accuracy: 0.9227390885353088
